########## choose stock list (aka market) ##################

In [57]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date

In [58]:
## To acquire stock list in NASDAQ ###
ex1 = pd.read_csv('stock_info_us_1.csv')
ex2 = pd.read_csv('stock_info_us_2.csv')
df = pd.DataFrame(ex1['Symbol'].astype('str'))
df = pd.concat([df,pd.DataFrame(ex2['Symbol'].astype('str'))])
df = df.rename(columns={'Symbol' : 'ticker'})
df = df.reset_index(drop=True)
df = df.drop_duplicates()
df = df.reset_index(drop=True)

# df = df.loc[df['ticker'] == 'AMLX']
# df = df.reset_index(drop=True)
df

,ticker
0,AACG
1,AADI
2,AAGR
3,AAGRW
4,AAL
...,...
6755,ZTR
6756,ZTS
6757,ZUO
6758,ZVIA


### Acquire informations from yfinance function >> .info

In [59]:
# 1st pulling date for this data #
df['date_pulling'] = date.today()

In [60]:
# 2nd acquiring information from .info #
info_attribute_list = [
    'industry',
    'sector',
    'enterpriseValue',
    'totalCashPerShare'
]
# 3rd acquiring information from .quarterly_balance_sheet #
balancesheet_list = [
    'Total Assets',
    'Current Liabilities'
]

# 4th acquiring information from .quarterly_financials #
ttm = 4
financials_list = [
    'EBIT',
    'Operating Income'
]

info_attribute_list_buffer = []
balancesheet_list_buffer = []
financials_list_buffer = []
for i in range(len(df)):
    # print(i, " ",df['ticker'][i])
    yfticker = yf.Ticker(df['ticker'][i])

    # 2nd acquiring information from .info #
    info_attribute_list_buffer_r = []
    for j in range(len(info_attribute_list)):
        try:
            info_attribute_list_buffer_r.append(yfticker.info[info_attribute_list[j]])
        except:
            info_attribute_list_buffer_r.append(None)
    info_attribute_list_buffer.append(info_attribute_list_buffer_r)

    # 3rd acquiring information from .balancesheet #
    balancesheet_list_buffer_r = []
    for j in range(len(balancesheet_list)):
        try:
            balancesheet_list_buffer_r.append(yfticker.quarterly_balance_sheet.loc[yfticker.quarterly_balance_sheet.index == balancesheet_list[j]].values[0][0])
        except:
            balancesheet_list_buffer_r.append(None)
    balancesheet_list_buffer.append(balancesheet_list_buffer_r)

    # 4th acquiring information from .quarterly_financials #
    financials_list_buffer_r = []
    for j in range(len(financials_list)):
        try:
            financials_list_buffer_r.append(np.array([yfticker.quarterly_financials.loc[yfticker.quarterly_financials.index == financials_list[j]].values[0][i] for i in range(ttm)]).sum())
        except:
            financials_list_buffer_r.append(None)
    try:
        financials_list_buffer_r.append(yfticker.quarterly_financials.columns[0])
    except:
        financials_list_buffer_r.append(None)
    financials_list_buffer.append(financials_list_buffer_r)

financials_list.append('ttm_latest')
df = df.join(pd.DataFrame(info_attribute_list_buffer, columns=info_attribute_list))
df = df.join(pd.DataFrame(balancesheet_list_buffer, columns=balancesheet_list))
df = df.join(pd.DataFrame(financials_list_buffer, columns=financials_list))


#### Drop NAN #############
df = df.dropna()
##### Drop where marketcap and EBIT < 0#############
df = df.loc[df['enterpriseValue'] >0]
df = df.loc[df['EBIT'] >0]
df = df.loc[df['Operating Income'] >0]
df = df.reset_index(drop=True)

df.to_csv('data_stock_us.csv',index= False)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABR%5ED?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABR%5ED&crumb=oj8JRHy%2FYbf
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABR%5EE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABR%5EE&crumb=oj8JRHy%2FYbf
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABR%5EF?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABR%5EF&crumb=oj8JRHy%2FYbf
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ACP%5EA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDet

############################################################################

In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date
df = pd.read_csv('data_stock_us.csv')
print('sector', df['sector'].unique())
print('industry', df['industry'].unique())

sector ['Industrials' 'Technology' 'Financial Services' 'Consumer Cyclical'
 'Healthcare' 'Communication Services' 'Utilities' 'Consumer Defensive'
 'Energy' 'Basic Materials' 'Real Estate']
industry ['Airlines' 'Building Products & Equipment' 'Consumer Electronics'
 'Insurance - Life' 'Travel Services' 'Biotechnology'
 'Medical Care Facilities' 'Software - Infrastructure'
 'Semiconductor Equipment & Materials' 'Business Equipment & Supplies'
 'Software - Application' 'Semiconductors' 'Electrical Equipment & Parts'
 'Entertainment' 'Utilities - Regulated Electric'
 'Education & Training Services' 'Conglomerates'
 'Medical Instruments & Supplies'
 'Drug Manufacturers - Specialty & Generic' 'Electronic Components'
 'Computer Hardware' 'Drug Manufacturers - General' 'Capital Markets'
 'Furnishings, Fixtures & Appliances' 'Internet Retail'
 'Food Distribution' 'Internet Content & Information' 'Oil & Gas E&P'
 'Trucking' 'Home Improvement Retail' 'Specialty Retail' 'Thermal Coal'
 'Solar' '

In [10]:

### Next I perform calculations for MF_ROC and MF_EY ###
# represents_earning = 'Operating Income'
represents_earning = 'EBIT'

df['MF_ROC'] = df[represents_earning]/(df['Total Assets'] - df['Current Liabilities'])
df['MF_EY'] = df[represents_earning]/df['enterpriseValue']
df

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY
0,AAL,2024-08-18,Airlines,Industrials,3.705987e+10,13.685,6.412500e+10,2.461000e+10,1.996000e+09,2.852000e+09,2024-06-30,0.050512,0.053859
1,AAON,2024-08-18,Building Products & Equipment,Industrials,7.222197e+09,0.035,1.010716e+09,1.413190e+08,2.431950e+08,2.426760e+08,2024-06-30,0.279728,0.033673
2,AAPL,2024-08-18,Consumer Electronics,Technology,3.476398e+12,4.065,3.316120e+11,1.316240e+11,1.216250e+11,1.205940e+11,2024-06-30,0.608161,0.034986
3,ABL,2024-08-18,Insurance - Life,Financial Services,8.722294e+08,1.277,4.838549e+08,5.538306e+07,1.505907e+07,1.895698e+07,2024-06-30,0.035146,0.017265
4,ABNB,2024-08-18,Travel Services,Consumer Cyclical,6.458017e+10,17.790,2.632000e+10,1.582800e+10,1.971000e+09,1.598000e+09,2024-06-30,0.187857,0.030520
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133,ZBH,2024-08-18,Medical Devices,Healthcare,2.815703e+10,2.063,2.152000e+10,3.608400e+09,1.224200e+09,1.507600e+09,2024-06-30,0.068347,0.043478
2134,ZIP,2024-08-18,Staffing & Employment Services,Industrials,8.540356e+08,5.268,6.652730e+08,7.577100e+07,7.541800e+07,5.447000e+07,2024-06-30,0.127935,0.088308
2135,ZTO,2024-08-18,Integrated Freight & Logistics,Industrials,1.258625e+10,24.325,8.987536e+10,2.326055e+10,1.087243e+10,1.032418e+10,2024-03-31,0.163213,0.863834
2136,ZTS,2024-08-18,Drug Manufacturers—Specialty & Generic,Healthcare,8.842400e+10,3.474,1.416500e+10,1.797000e+09,3.129000e+09,3.214000e+09,2024-06-30,0.252992,0.035386


In [11]:
### Drop Utilities, Energy, and Financial Services  as suggested from the book ###
sectortoexclude = [
    'Utilities',
    'Energy',
    'Financial Services',
    'Real Estate'
]
for i in sectortoexclude:
    print('exclude sector > ',i)
    try:
         
        df = df.loc[df['sector'] != i]
    except:
        None
        
industrytoexclude =[
    'Engineering & Construction',
    'Building Products & Equipment',
    'Building Materials'
]
for i in industrytoexclude:
    print('exclude industry > ',i)
    try:
         
        df = df.loc[df['industry'] != i]
    except:
        None

df = df.reset_index(drop=True)

exclude sector >  Utilities
exclude sector >  Energy
exclude sector >  Financial Services
exclude sector >  Real Estate
exclude industry >  Engineering & Construction
exclude industry >  Building Products & Equipment
exclude industry >  Building Materials


In [12]:
#### Select market size by choosing ' market ' ###
market =  50000000### in USD
df_market = df.loc[df['enterpriseValue'] >= market]
df_market = df_market.reset_index(drop=True)

In [13]:
### Ranking regarding MFs####

df_market['Ranking_MF_ROC'] = df_market['MF_ROC'].rank()
df_market['Ranking_MF_EY'] = df_market['MF_EY'].rank()
df_market['Ranking_MF'] = df_market['Ranking_MF_ROC'] + df_market['Ranking_MF_EY']

df_market.loc[df_market['Ranking_MF'] == df_market['Ranking_MF'].max()]

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
345,JFIN,2024-08-18,Internet Content & Information,Communication Services,777966976.0,2.678,5.160597e+09,2.004158e+09,1.299660e+09,1.299660e+09,2024-03-31,0.411749,1.670585,1478.0,1518.0,2996.0


In [14]:
numstocks = 50
df_sorted = df_market.sort_values(by=['Ranking_MF'],ascending=False)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted[:numstocks]

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,JFIN,2024-08-18,Internet Content & Information,Communication Services,7.779670e+08,2.678,5.160597e+09,2.004158e+09,1.299660e+09,1.299660e+09,2024-03-31,0.411749,1.670585,1478.0,1518.0,2996.0
1,GDEN,2024-08-18,Resorts & Casinos,Consumer Cyclical,1.260818e+09,3.130,1.181221e+09,1.693220e+08,4.187260e+08,6.473400e+07,2024-06-30,0.413802,0.332107,1480.0,1493.0,2973.0
2,RMNI,2024-08-18,Software—Application,Technology,1.132910e+08,1.480,3.673690e+08,3.032360e+08,3.000900e+07,3.953700e+07,2024-06-30,0.467918,0.264884,1490.0,1480.0,2970.0
3,PROC,2024-08-18,Drug Manufacturers—Specialty & Generic,Healthcare,4.715110e+08,0.174,4.985540e+08,2.029300e+08,1.289630e+08,3.238400e+07,2023-09-30,0.436240,0.273510,1485.0,1485.0,2970.0
4,NVO,2024-08-18,Biotechnology,Healthcare,5.957051e+11,14.055,3.693830e+11,1.879090e+11,1.166360e+11,1.114590e+11,2024-06-30,0.642715,0.195795,1508.0,1456.0,2964.0
5,KARO,2024-08-18,Software—Application,Technology,4.906297e+08,32.242,4.565444e+09,1.055226e+09,1.155279e+09,1.117875e+09,2024-05-31,0.329119,2.354686,1433.0,1519.0,2952.0
6,REVG,2024-08-18,Farm & Heavy Construction Machinery,Industrials,1.720201e+09,0.736,1.330400e+09,5.185000e+08,3.462000e+08,1.043000e+08,2024-04-30,0.426407,0.201256,1482.0,1461.0,2943.0
7,BWMX,2024-08-18,Specialty Retail,Consumer Cyclical,5.444902e+09,11.364,1.101829e+10,3.860136e+09,2.397458e+09,2.377988e+09,2024-06-30,0.334927,0.440312,1439.0,1503.0,2942.0
8,SIGA,2024-08-18,Drug Manufacturers—Specialty & Generic,Healthcare,6.433619e+08,1.499,1.926586e+08,2.137805e+07,1.027231e+08,1.027231e+08,2024-06-30,0.599736,0.159666,1506.0,1426.0,2932.0
9,OMAB,2024-08-18,Airports & Air Services,Industrials,3.498960e+10,4.169,2.590749e+10,4.752860e+09,7.834555e+09,7.527290e+09,2024-06-30,0.370347,0.223911,1456.0,1468.0,2924.0


In [15]:
df_sorted.loc[df_sorted['ticker'] == 'ASO']

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
107,ASO,2024-08-18,Specialty Retail,Consumer Cyclical,5.270136e+09,5.243,4.676713e+09,879858000.0,709207000.0,677855000.0,2024-01-31,0.186788,0.134571,1200.0,1385.0,2585.0


In [16]:
df_sorted_2 = df_sorted[:numstocks].copy()
df_sorted_2 = df_sorted_2.sort_values(by=['totalCashPerShare'],ascending=False)
df_sorted_2 = df_sorted_2.reset_index(drop=True)
df_sorted_2

,ticker,date_pulling,industry,sector,enterpriseValue,totalCashPerShare,Total Assets,Current Liabilities,EBIT,Operating Income,ttm_latest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,TLK,2024-08-18,Telecom Services,Communication Services,6.996729e+13,283.892,2.859920e+14,8.317300e+13,4.434200e+13,4.457200e+13,2024-06-30,0.218628,0.633753,1292.0,1510.0,2802.0
1,TSM,2024-08-18,Semiconductors,Technology,3.534018e+12,79.010,5.787891e+12,1.026180e+12,1.082195e+12,1.023834e+12,2024-06-30,0.227270,0.306222,1304.0,1490.0,2794.0
2,DDS,2024-08-18,Department Stores,Consumer Cyclical,5.212588e+09,66.074,3.937156e+09,1.130290e+09,9.238860e+08,8.957790e+08,2024-04-30,0.329152,0.177241,1434.0,1446.0,2880.0
3,ASR,2024-08-18,Airports & Air Services,Industrials,8.908710e+10,49.990,7.299319e+10,4.898742e+09,1.674405e+10,1.580910e+10,2024-03-31,0.245894,0.187951,1345.0,1451.0,2796.0
4,PDD,2024-08-18,Internet Retail,Consumer Cyclical,5.898885e+11,43.947,3.778261e+11,1.513980e+11,9.543905e+10,7.774340e+10,2024-03-31,0.421498,0.161792,1481.0,1429.0,2910.0
5,KARO,2024-08-18,Software—Application,Technology,4.906297e+08,32.242,4.565444e+09,1.055226e+09,1.155279e+09,1.117875e+09,2024-05-31,0.329119,2.354686,1433.0,1519.0,2952.0
6,IDCC,2024-08-18,Software—Application,Technology,3.113497e+09,30.257,1.636295e+09,6.993470e+08,3.677850e+08,3.166220e+08,2024-06-30,0.392535,0.118126,1467.0,1338.0,2805.0
7,AMR,2024-08-18,Coking Coal,Basic Materials,2.803547e+09,25.826,2.492952e+09,3.086540e+08,5.282050e+08,5.430350e+08,2024-06-30,0.241819,0.188406,1340.0,1452.0,2792.0
8,PAC,2024-08-18,Airports & Air Services,Industrials,1.152916e+11,24.907,7.331772e+10,1.631331e+10,1.559022e+10,1.458257e+10,2024-06-30,0.273491,0.135224,1381.0,1386.0,2767.0
9,JBL,2024-08-18,Electronic Components,Technology,1.284190e+10,21.887,1.744900e+10,1.142300e+10,1.990000e+09,1.520000e+09,2024-05-31,0.330236,0.154962,1435.0,1417.0,2852.0
